In [14]:
import spacy
import pandas as pd

In [15]:
nlp = spacy.load("fr_core_news_md")

In [33]:
#Lit le fichier des gares et filtrent sur les gares qui accueillent des voyageurs
liste_gare = pd.read_csv("./liste-des-gares.csv", sep=';')
liste_gare = liste_gare.loc[liste_gare["VOYAGEURS"] == "O"]

#Lit le jeu de tests
liste_phrase_test = pd.read_csv("./liste-de-phrases.csv", sep=";")

#Applique des filtres sur la liste des gares pour une analyse approfondie plus tard par l'ia
gares = list(liste_gare["COMMUNE"])
gares = [x.lower() for x in gares]
gares_without_starting = [x[2:] for x in gares if x[0:2] == "l'"]
gares_without_dash = [x.replace("-", " ") for x in gares if "-" in x]
gares = [x[2:] if x[0:2] == "l'" else x for x in gares]

3884
3352


In [17]:
gares_without_starting

['hermitage',
 'isle-jourdain',
 "hospitalet-pres-l'andorre",
 'etang-la-ville',
 'aigle',
 'isle-sur-le-doubs',
 'escarene',
 'isle-sur-la-sorgue',
 'etang-la-ville',
 "isle-d'abeau",
 'argentiere-la-bessee',
 'etang-la-ville',
 'hopital-du-grosbois']

In [18]:
#Liste des prépositions et verbe nécessaire à l'analyse sémantique
preposition_depart = ["de", "depuis", "à", "sur", "dans", "vers"]
preposition_destination = ["à", "pour", "sur", "dans", "jusque", "jusqu'à", "vers", "de"]
preposition_in_both = ["de", "à", "sur", "dans", "vers"]
preposition = set(preposition_depart + preposition_destination)
verbe_action = ["aller", "partir", "rendre", "rejoindre", "retourner"]
verbe_potentiel_action = ["vouloir", "souhaiter", "essayer", "devoir", "pouvoir", "aimer"]

In [19]:
#Liste de la ponctuation
ponctuation_list = [".", ",", ";", ":", "!", "?"]

In [20]:
text = "Je souhaite visiter un musée de Paris, je pars de strasbourg"
def get_main_data(text):
    """
        Retourne une analyse sémantique de la phrase passé en paramètre, elle retourne un array contenant:
            - un array avec les labels de chaque mot de la forme [mot, label]
            - la première ville trouvée avec la préposition qui l'accompagne si trouvé: [ville] ou [préposition, ville]
            - la deuxième ville trouvée avec la préposition qui l'accompagne si trouvé: [ville] ou [préposition, ville]
    """
    #Initialisation des variables
    doc = nlp(text)
    position = []
    loc_1, loc_2, nb_loc = [], [], 0

    #Analyse sémantique de la phrase
    for w in doc:
        w_text = w.text
        if w.pos_ == "ADP" and w_text in preposition:
            position.append([w_text, w.pos_])
        if w_text in gares:
            if nb_loc == 0:
                if position and position[-1][1] == "ADP":
                    loc_1.append(position[-1])
                loc_1.append([w_text.upper(), "LOC"])
                nb_loc += 1
            else:
                if position and position[-1][1] == "ADP":
                    loc_2.append(position[-1])
                loc_2.append([w_text.upper(), "LOC"])
            position.append([w_text.upper(), "LOC"])
        if w.pos_ == "VERB" and w.lemma_ in verbe_action:
            position.append([w.lemma_, "ACTION"])
        if w.pos_ == "VERB" and w.lemma_ in verbe_potentiel_action:
            position.append([w.lemma_, "POT_ACTION"])
    return [position, loc_1, loc_2]
#print([(w.text, w.label_) for w in doc.ents])
get_main_data(text)

[[['souhaiter', 'POT_ACTION'],
  ['de', 'ADP'],
  ['partir', 'ACTION'],
  ['de', 'ADP'],
  ['STRASBOURG', 'LOC']],
 [['de', 'ADP'], ['STRASBOURG', 'LOC']],
 []]

In [21]:
def is_departure(loc, loc_2):
    """
        vérifie si une ville est la ville de départ dans une phrase
    """
    if loc[0][1] == "ADP":
        if loc[0][0] not in preposition_in_both:
            if loc[0][0] in preposition_depart:
                return True
    elif loc[0][1] == "LOC" and len(loc_2) == 1:
        return True
    return False

In [22]:
def get_city_name(loc_1, loc_2):
    """
        Récupère le nom de la ville dans un array qui peut prendre la forme [Préposition, ville] ou [ville] 
    """
    city_1, city_2 = "", ""
    if len(loc_1) == 2:
        city_1 = loc_1[1][0]
    else:
        city_1 = loc_1[0][0]

    if len(loc_2) == 2:
        city_2 = loc_2[1][0]
    else:
        city_2 = loc_2[0][0]
    return [city_1, city_2]

In [23]:
def is_de_a(loc_1, loc_2):
    """
        Vérifie si on se retrouve dans le cas d'une phrase avec 2 villes, et les prépositions "de, à" devant le nom des villes
    """
    prep1 = loc_1[0][0]
    prep2 = loc_2[0][0]
    if prep1 in ["de", "à"] and prep2 in ["de", "à"]:
        return True
    return False

In [24]:
def get_position(text):
    """
        Retourne les villes trouvées dans le sens départ, destination ou une erreur si ce n'est pas possible
    """
    #Prétraitement des données pour une meilleur analyse
    text = text.lower()

    #On retire toute ponctuation parce qu'elles ne sont pas utiles
    for ponctu in ponctuation_list:
        text = text.replace(ponctu, "")
    
    #On recherche les noms de gares qui ont un nom qui peut poser des problèmes 
    for city in gares_without_dash:
        if city in text:
           text = text.replace(city, city.replace(" ", "-"))
    
    #On fait une première analyse sémantique pour récupérer des infos essentielles, voir fonction get_main_data() 
    [data, loc_1, loc_2] = get_main_data(text)

    #Actuellement, si on a pas 2 villes on retourne une erreur
    if not loc_1 or not loc_2:
        return ["Il manque des villes ou des villes n'ont pas été détecté. Il se peut aussi que la ville n'a pas été détecté parce qu'il n'y a pas de gare"]
    
    depart, destination = "", ""

    #if, elif permettent de chercher des cas simples sémantiquement et gagner du temps de travail, pour les cas plus complexes voir le else
    if is_departure(loc_1, loc_2):
        if len(loc_1) == 2:
            depart = loc_1[1][0]
        else:
            depart = loc_1[0][0]
        if len(loc_2) == 2:
            destination = loc_2[1][0]
        else:
            destination = loc_2[0][0]
    elif is_departure(loc_2, loc_1):
        if len(loc_2) == 2:
            depart = loc_2[1][0]
        else:
            depart = loc_2[0][0]
        if len(loc_1) == 2:
            destination = loc_1[1][0]
        else:
            destination = loc_1[0][0]
    elif is_de_a(loc_1, loc_2):
        if loc_1[0][0] == "de":
            depart = loc_1[1][0]
            destination = loc_2[1][0]
        else:
            depart = loc_2[1][0]
            destination = loc_1[1][0]
    else:
        pot_action = False
        action = False
        
        #Analyse est divisé en 2 partie, la première est ce que j'appelle 
        # le potentiel d'action ou pot_action, qui a pour but d'analyser l'intention d'une phrase à l'aide de sa sémantique
        # par exemple: je souhaite aller à Paris, grâce au verbe souhaiter, on sait que la personne "souhaite"(à l'intention de) se rendre à Paris
        for [w, label] in data:
            if label == "POT_ACTION":
                pot_action = True
                pass
            
            if pot_action:
                if label == "LOC":
                    destination = w
                    cities = get_city_name(loc_1, loc_2)
                    for city in cities:
                        if city != w:
                            depart = city
                    break
        
        #si il n'y a pas d'intention dans la phrase, on passe à la deuxième partie d'analyse, 
        # l'action, on va chercher à savoir si sémantiquement une action est déclaré,
        #  par exemple: je vais à paris. grâce au verbe aller on sait qu'une action va se produire ou se déroule
        if not destination:
            for [w, label] in data:
                if label == "ACTION":
                    action = True
                    pass
                if action:
                    if label == "LOC":
                        destination = w
                        cities = get_city_name(loc_1, loc_2)
                        for city in cities:
                            if city != w:
                                depart = city
                        break
    #Retourne le résultat de l'analyse en faisant une transformation sur les données par la même occasion pour les villes spéciales
    return ["L'" + x if x.lower() in gares_without_starting else x for x in [depart, destination]]
get_position(text)

['PARIS', 'STRASBOURG']

In [25]:
#Process de tests pour calculer les résultats de l'IA
nb_passed, nb_no_passed = 0, 0

for i, data in liste_phrase_test.iterrows():
    sentence = data["phrase"]
    destination = data["destination"]
    depart = data["départ"]
    
    if [depart.upper(), destination.upper()] == get_position(sentence):
        nb_passed += 1
    else:
        print(sentence)
        [data, loc_1, loc_2] = get_main_data(sentence)
        print(loc_1)
        print(loc_2)
        nb_no_passed += 1

print("Nombre passé: ", nb_passed)
print("Nombre non passé: ", nb_no_passed)
print("Nombre total: ", nb_passed + nb_no_passed)


Je souhaite visiter un musée de Paris, je pars de strasbourg
[['de', 'ADP'], ['STRASBOURG', 'LOC']]
[]
Je vais visiter un musée de paris, je pars de strasbourg
[['de', 'ADP'], ['PARIS', 'LOC']]
[['de', 'ADP'], ['STRASBOURG', 'LOC']]
Je suis à strasbourg et je retourne à paris
[['à', 'ADP'], ['STRASBOURG', 'LOC']]
[['à', 'ADP'], ['PARIS', 'LOC']]
Je veux passer par Lyon pour aller à Strasbourg en partant de Paris
[]
[]
Nombre passé:  25
Nombre non passé:  4
Nombre total:  29


In [26]:
#Permet de développer l'analyse d'une phrase qui ne serait pas passer
text_test = "Je souhaite visiter un musée de Paris, je pars de strasbourg"
text_test = text_test.lower()
for ponctu in ponctuation_list:
    text_test = text_test.replace(ponctu, "")
doc = nlp(text_test)
print([(w.text, w.pos_) for w in doc])

get_main_data(text_test)

[('je', 'PRON'), ('souhaite', 'VERB'), ('visiter', 'VERB'), ('un', 'DET'), ('musée', 'NOUN'), ('de', 'ADP'), ('paris', 'PROPN'), ('je', 'PRON'), ('pars', 'VERB'), ('de', 'ADP'), ('strasbourg', 'PROPN')]


[[['souhaiter', 'POT_ACTION'],
  ['de', 'ADP'],
  ['PARIS', 'LOC'],
  ['partir', 'ACTION'],
  ['de', 'ADP'],
  ['STRASBOURG', 'LOC']],
 [['de', 'ADP'], ['PARIS', 'LOC']],
 [['de', 'ADP'], ['STRASBOURG', 'LOC']]]